# The `swifttools.ukssdc.data.GRB` module

## Summary

This module provides direct access to the [GRB products created at the UKSSDC](https://www.swift.ac.uk/xrt_products), as well as the means to manipulate them (e.g. rebinning light curves). Being part of the [`swifttools.ukssdc.data` module](../data.ipynb) it is designed for use where you already know which GRB(s) you want to get data for. If you don't know that, for example you want to select them based on their characteristics, you will need the [GRB query module](../query/GRB.ipynb).

**Tip** You can read this page straight through, and you may want to, but there is a lot in it. So if you want a full overview, read through. If you want to know how to do do something specific, I'd advise you to read the introduction, and then use the [contents links](#page-contents) to jump straight to the section you are interested in.

You will notice that the functions in this module all begin with `get` (whereas the parent module used `download`). As you'll realise pretty quickly, this is because this module lets you do more than just download datafiles, you can instead pull the data into variables for example.

An important point to note is that this documentation is about the API, not the GRB products. So below we will talk about things like WT mode systematic errors, and unreliable light curve data, without any real explanation; you can read the documentation for the products if you don't understand.

OK, first let's import the module, using a short form to save our fingers:

In [ ]:
import swifttools.ukssdc.data.GRB as udg

After a short introduction, this page is split into sections according to the product type as below.

## Page contents

* [Introduction and common behaviour](#intro)
* [Light curves](#lightcurves)
  * [Rebinning](#rebin)
* [Spectra](#spectra)
  * [Time slicing](#slice)
* [Burst analyser](#ban)
* [Positions](#positions)
* [Obs data](#obs)

----

<a id='intro'></a>
## Introduction and common behaviour

Before we dive into things, let me introduce a couple of important concepts which are common to everything that follows.

Firstly, you specify which GRB(s) you are interested in either by their name, or their targetID. You do this (prepare for a shock) by supplying the `GRBName` or `targetID` argument to the relevant function: all functions in this module to get data take these arguments. You must supply one or the other though, if you suply a `GRBName` and `targetID` you will get an error. These arguments can be either a single value (e.g. `GRBName="GRB 060729"`) or a list/tuple (e.g. `targetID=[282445, 221755]`) if you want more than one GRB. Hold that thought for just a moment because we'll come back to it in a tick.

The second common concept is how the data you request are stored. This is controlled by two arguments, which again are common to all of the data-access functions in this module. These are:

* `returnData` - (default `False`), whether or not the function should return a `dict` containing the data.
* `saveData` - (default: `True`), whether or not the function should download the data files from the UKSSDC website and write them to disk.

You can set both to `True` to return and save. (You can also set both to `False` if you like wasting time, CPU and making TCP packets feel that they serve no real purpose in life, but why would you do that?) If you are using `saveData=True` you may also need the `clobber` parameter. This specifies whether existing files should be overwritten, and is `False` by default.

You should still be holding onto a thought&hellip; the fact that you can supply either a single value or a list (or tuple) to identify the GRBs you want. What you do here affects how the data are stored as well.

If you gave a single value, then `saveData=True` will cause the data for your object to be downloaded to the specified directory. `returnData=True` will cause the function to return a `dict` containing your data.

If you supplied a list of objects then of course you're going to get multiple dataset. `saveData=True` will (by default) create a subdirectory per GRB to stick the data in. `returnData=True` will return a `dict` with an entry per GRB; that entry will the `dict` containing the data.

But, you may ask, what will the names of these subdirectories, or the keys of these `dict`s be? They will be whatever identifer you used to specify the GRBs you wanted. If you used the `GRBName` argument, the directories and `dict` keys will be the GRB names; if you used `targetID` then they will be, er, the target IDs.

That may not make so much sense in the abstract, but all will become clear as we see some demos. Oh, but just to further confuse you, please do note that a list (or tuple) with one entry is still a list (unless it's a tuple), and so the returned data will still have the subdirectories/extra `dict` that you get when you supply a list (or tuple) even though there is only one entry in this. Confused? Me too, but it will make sense when we start playing, so let's do so.

<a id='lightcurves'></a>
## Light curves

There are basically three ways we can play with light curve data, let's start with just downloading them straight from the UKSSDC website.

### Saving directly to disk

Our function for getting light curves is cunningly named `getLightCurves()`. As I'm sure you remember from above, it we want to save the data to disk, we supply `saveData=True` (this is the default but I think it's helpful to be explicit). Let's try that:

In [ ]:
udg.getLightCurves(GRBName="GRB 060729", saveData=True, destDir="/tmp/APIDemo_GRBLC1", silent=False)

In the above I turned `silent` off just to show you that something was happening; feel free to go and look at '/tmp/APIDemo_GRBLC1' to see what was downloaded.

`getLightCurves()` mainly just wraps the common `getLightCurve()` function, so see [its documentation](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#getlightcurve) for a description of the various arguments, but there is one extra argument available to you to mention here: `subDirs`.

This variable, a boolean which defaults to `True`, only matters if you supplied a list/tuple of GRBs. If it is `True` then a subdirectory will be created for each GRB (as I mentioned [above](#intro). However, you can set this to `False`, if you want to put all the data in the same directory. In this case the GRB name or targetID (depending on which argument you called `getLightCurves()` with) will be prepended to the file names.

Let's demonstrate this with a couple of simple downloads:

In [ ]:
lcData = udg.getLightCurves(GRBName=("GRB 060729", "GRB 080319B"), destDir="/tmp/APIDemo_GRBLC2", silent=False)

And as you can see, and hopefully expected, the GRBs were each saved into their own directory. Now let's do exactly the same thing, but disable the subdirs:

In [ ]:
lcData = udg.getLightCurves(
    GRBName=("GRB 060729", "GRB 080319B"), destDir="/tmp/APIDemo_GRBLC3", subDirs=False, silent=False, verbose=True
)

I turned `verbose` on as well so that you could see what was happening: the data all got saved into the same directory, with the GRB name prepended to the files.

In these examples I've given the GRB name, but the targetID was an option if you happen to know it:

In [ ]:
lcData = udg.getLightCurves(targetID=(221755, 306757), destDir="/tmp/APIDemo_GRBLC4", silent=False, verbose=True)

I trust this hasn't surprised you.

As a final demonstration, let's illustrate the point that a tuple with one entry is still a tuple, and so the way the data are saved is set accordingly:

In [ ]:
lcData = udg.getLightCurves(GRBName=("GRB 060729",), destDir="/tmp/APIDemo_GRBLC5", silent=False, verbose=True)

As you can see here, we only got a single GRB, but because we supplied its name as a tuple, not as a string, the data were saved into a subdirectory just as if we'd requested multiple GRBs.

### Storing the light curves in variables

Let's move on now to the `returnData=True` case. As I told you [earlier](#intro) this will return a `dict` containing the data. All light curves returned by anything in the `swifttools.ukssdc` module have a common structure which I call a "light curve `dict`", and you can [read about this here](https://www.swift.ac.uk/API/ukssdc/structures.md#the-light-curve-dict).

There are no special parameters related to returning data, so let's jump straight in with some demonstrations similar to those above.

In [ ]:
lcData = udg.getLightCurves(GRBName="GRB 220427A", incbad="both", nosys="both", saveData=False, returnData=True)

I don't recommend simply printing `lcData` straight, it's quite big. If you [read about the light curve `dict`](https://www.swift.ac.uk/API/ukssdc/structures.md#the-light-curve-dict) then you may have an idea what to expect, but it's helpful to explore it anyway, so let's do that.

In [ ]:
list(lcData.keys())

(I used `list()` above because Jupyter renders it a bit more nicely than the `dict_keys` object). There is a lot to take in there, but most of those entries are just light curve data.

Let's first just check the keys that gave me some information about the light curve generically:

In [ ]:
print(f"Binning: {lcData['Binning']}")
print(f"TimeFormat: {lcData['TimeFormat']}")
print(f"T0: {lcData['T0']}")

So we know that the light curves are binned by the number of counts per bin (no surprise, this is standard for GRBs), the timeformat is in Swift Mission Elapsed Time, and is in seconds since MET 672786064 (again, see the [light curve `dict` documentation](https://www.swift.ac.uk/API/ukssdc/structures.md#the-light-curve-dict) for more info).

The 'URLs' key lists the URL to the individual files in the light curve by dataset. I'm not going to print it here because it's long (and boring) but you can explore it if you want.

The 'Datasets' key is really the crucial one for exploring the data, it's essentially an index of all the datasets we obtained, in the form of a list. Let's take a look:

In [ ]:
lcData["Datasets"]

As a quick aside, you may wonder why we bother with this array, but we can't just step over the keys in `lcData` - 'Binning', for example, is not a light curve, and maybe in the future we'll want to add other things, so 'Datasets' is handy. 

There are a lot of datasets in this example, because we set both `incbad` and `nosys` to "both", so we got all data with/out missing centroids and with/out WT-mode systematics (if you don't know what I'm talking about, see the [the light curve documentation](https://www.swift.ac.uk/user_objects/lc_docs.php#systematics).


The contents of the datasets were discussed in [light curve `dict` documentation](https://www.swift.ac.uk/API/ukssdc/structures.md#the-light-curve-dict) (I'm sounding like a broken record, I know), so I'm not going to spend time on it here, except to show you one entry as an example:

In [ ]:
lcData["PC"]

(If you're reading the markdown version, I have probably truncated the output, removed some of the rows).

OK, so that's what `returnData=True` does. If we supply a list of GRBs then, as you should expect, we get a `dict` of light curves `dict`s; the top level is indexed either by GRB name, or targetID, depending on how you called the function, so:


In [ ]:
lcData = udg.getLightCurves(GRBName=("GRB 220427A", "GRB 070616"), saveData=False, returnData=True)

In [ ]:
list(lcData.keys())

I trust this doesn't come as a surprise! Nor should the fact that each of these in turn is a light curve `dict` similar to that above (although this time I left `nosys` and `incbad` to their defaults). I can prove this easily enough:

In [ ]:
list(lcData["GRB 070616"].keys())

(I'll let you explore the other GRB yourself).

If we had supplied `targetID=(1104343, 282445)` you would have got the same data, but indexed by these targetIDs, not the name. You can test it out if you don't trust me, but frankly, I'm expecting most people know GRBs by their names not the Swift targetIDs!

#### Plotting light curves

If we've downloaded a light curve then we can make use of the [module-level `plotLightCurve()` function](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#plotlightcurve) to give us a quick plot. I'm not going to repeat the `plotLightCurve()` documentation here, but I will note that its first argument is a single light curve `dict` so if, as in our case here, we downloaded multiple GRB light curves, we have to provide one of them to the function, like this:

In [ ]:
from swifttools.ukssdc import plotLightCurve

fig, ax = plotLightCurve(lcData["GRB 070616"], whichCurves=("WT_incbad", "PC_incbad"), xlog=True, ylog=True)

You'll note I captured the return in variable names which will be familiar to users of `pyplot` and can be ignored by everyone else because you'll need to be familiar with `pyplot` to take advantage of the fact that `plotLightCurve` returnes them for you.

### The third way - save from a variable

We've covered two obvious ways of getting light curve data: downloading them straight to disk or to variables. But there is a third way (and it does not involve steep stairs or giant spiders): you pull the data into variables as above, and then save them to disk from there (OK, I guess maybe it's a two-and-a-halfth way, but that doesn't scan and can't be linked to Tolkien).

To do this we use the function `saveLightCurves()`, and the reason for providing this option as well as the `saveData=True` option above is that this gives you some more control over how the data are saved.

This function actually doesn't do very much itself, most of the work is done by another common function: [`saveLightCurveFromDict()`](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#savelightcurve), and most of the arguments you may pass to `saveLightCurves()` are just keyword arguments that will get passed straight through, but there are a few things about `saveLightCurves()` to note here.

First, you may have spotted that the function name contains lightCurve**s** plural, whereas the common function is singular; that is because `saveLightCurves()` allows you to save more than one light curve in a single command - if you retrieved more than one of course! This means it has two extra parameters for use when you supplied a list or tuple to `getLightCurves()`. We will discuss those parameters in a moment.

The second thing is that it will override the default values for the `timeFormatInFname` and `binningInFname` arguments for `saveLightCurveFromDict()`, setting them both to ``False`` unless you explicitly specify them. This is because GRB light curves are always in MET and counts per bin.


So, the parameters for `saveLightCurves()` are:

* `lcData` - The light curves to save; basically whatever `getLightCurves(returnData=True)` returned.
* `whichGRBs` - An optional list of which GRBs' light curves to save. The entries in this list should be keys in `lcData`, or you'll get an error. If you want to save all the GRBs, you can set `whichGRBs='all'`, which is the default.
* `subDirs` - A `bool` indicating whether the light curves for each GRB should be saved into separate subdirectories (which will be named for the key the GRB is indexed under in `lcData`). If `False`, then the file names will have the GRB key prepended to their file name, because otherwise all of the files will have the same name, and I trust you can spot the problem with that (default: `True`).

One quick note, if `lcData` is just a light curve `dict`, e.g. you called `getLightCurves()` with a single GRB name or targetID, not a list/tuple, then `subDirs` is ignored.

Right: a little less talk, a little more action is called for now. I'm going to give a new `getLightCurves()` call first, so this example is standalone.

In [ ]:
lcData = udg.getLightCurves(
    GRBName=("GRB 220427A", "GRB 070616", "GRB 080319B", "GRB 130925A"), saveData=False, returnData=True
)

And now let's demonstrate saving things. I will only save two of these, and I'll also only save a couple of datasets. Oh and just to demonstrate that you can, I will set the column separator to some custom value.

In [ ]:
udg.saveLightCurves(
    lcData,
    destDir="/tmp/APIDemo_GRBLC6",
    whichGRBs=("GRB 070616", "GRB 080319B"),
    whichCurves=("WTHR_incbad", "PCHR_incbad"),
    sep=";",
    verbose=True,
)

If you are looking at this and thinking, "Where did those arguments, `whichCurves`, `sep`, and `verbose` come from, the answer is of course that they are arguments taken by `saveLightCurveFromDict()`, as detailed in [that function's documentation](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#savelightcurve).

---

<a id='rebin'></a>
## Rebinning light curves

Everything detailed above was about getting at the automated GRB light curves. But, if you are an afficionado of [our website](https://www.swift.ac.uk/xrt_curves) then you will know that from there you can do more than just get the automated results, you can rebin them too. Wouldn't it be nice if you could rebin them via the Python API? Luckily for you, I'm nice (sometimes).

Actually, rebinning appears in a few places so [it is one of the common functions](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#rebinlightcurve). This module (`swifttools.ukssdc.data.GRB`, in case you've forgotten) provides its own `rebinLightCurve()` function, which requires **either** the `GRBName` or `targetID` parameter (as everything in this module); all the other arguments are passed straight to [the common function](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#rebinlightcurve).

One note before we give an example, for this function `GRBName` and `targetID` can ONLY be single values, not lists (or tuples); this is because the function sends a job request to our servers, and we don't want you accidentally overloading our servers with 300 jobs (we don't want you deliberately doing it either).

Right, let's plunge in with a demo, rebinning a GRB by one bin per observation, and asking for MJD on the time axis.

In [ ]:
JobID = udg.rebinLightCurve(GRBName="GRB 070616", binMeth="obsid", timeFormat="MJD")

That was easy enough. We'll unpack the arguments in a moment, but let's follow this example through to the end first. First, note that the function returned some identifier which we captured in the `JobID` variable. This really is critical because it's the only way that we can actually access our rebin request.

Data are not rebinned instantaneously, so we need to see how it's getting on. There are a couple of ways we can do this:

In [ ]:
udg.checkRebinStatus(JobID)

This function returns a `dict` telling you how things were going. On my computer it's telling me that the status 'running', but if you're running this notebook yourself, you may have a different status. Of course, you may not care about the status and just want to know if it's complete. You can do this either by knowing that the 'statusCode' will be 4 (and the text 'Complete'), or bypass that and call `rebinComplete()` which gives a simple `bool` (`True` meaning, "Yes, it's complete").

In [ ]:
udg.rebinComplete(JobID)

If the above is not `True`, then give it 30 seconds and try again, rinse and repeat until it is `True`, because the next steps will fail if the job hasn't completed.

OK, complete now? Great!

To get the light curve we use the function `getRebinnedLightCurve()` function. This again asks the common `getLightCurve()` function to do all the work, so if you didn't do so earlier (or jave jumped straight to the rebin docs) you should read [its documentation](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#getlightcurve). If you've worked through this notebook to this point, the only thing really to point out is that we don't specify which GRB to get, instead we tell it which rebin request we want the results of, by passing in the JobID. In this example, I will grab the data in a variable, rather than saving to disk (remember, you can do both, they're not mutually exclusive).

In [ ]:
lcData = udg.getRebinnedLightCurve(JobID, saveData=False, returnData=True)
list(lcData.keys())

As you can see, this is a light curve `dict` just like earlier. I hope that doesn't surprise you. For the sake of sanity (or, as I write this, one last-minute test), let's confirm that the binning method and time format of this new light curve are what I asked for:

In [ ]:
print(f"Binning: {lcData['Binning']}")
print(f"TimeFormat: {lcData['TimeFormat']}")

OK phew!

The only other thing to introduce here is the fact that you can cancel a rebin job if you change your mind or submitted it accidentally:

In [ ]:
udg.cancelRebin(JobID)

This returns a `bool` telling you whether the job was successfully cancelled or not. If it failed, as in this example, this is usually because the job had already completed, which we can check as above:

In [ ]:
udg.checkRebinStatus(JobID)

And a last note: the functions `checkRebinStatus()`, `rebinComplete()`, `getRebinnedLightCurve()` and `cancelRebin()` are really common functions, and technically should have been documented with the common functions, but it really makes no sense to demonstrate rebinning here without these functions.

Right, that's it for light curves. Let's move on to spectra.

----

<a id='spectra'></a>
## Spectra

We get spectra with the `getSpectra()` function. This is actually **not** a common function, even though the name is reused in other places -- the specifics of the use and arguments in each place are so different the functions are not conflated. 

I'll follow the same pattern in this section as I did for light curves, but don't worry if you haven't read that section, this one is intended to be standalone.


### Saving directly to disk.

As already discussed in [the introduction](#intro) (which I *am* assuming you've read), if you want to get a product by downloading files straight to disk, you use the `saveData=True` argument, and while this is the default, I think it's much more helpful to be explicit.

In [ ]:
udg.getSpectra(
    GRBName="GRB 130925A",
    saveData=True,
    saveImages=True,
    destDir="/tmp/APIDemo_GRB_Spec1",
    extract=True,
    removeTar=True,
    silent=False,
    verbose=True,
)

I turned on `verbose` mode to help you see what's going on here, but you may be wondering what all of those arguments were. `GRBName` and `saveData` were introduced in the, er, [introduction](#intro), and `silent` and `verbose` have been introduced [on the front page](https://www.swift.ac.uk/API/ukssdc/README.md). The other parameters all belong to the common, module-level function, `saveSpectrum()`, which is [documented here](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#savespectrum). You can probably guess at what these did from the output above - the images of the spectra were downloaded, as the `tar` archives of the actual spectral data. The latter were also extracted, and then removed.

I will demonstrate here one argument of this common function: the ability to choose which spectra get saved to disk. You may have noticed - and if you're familiar with the XRT GRB spectra it won't surprise you - there were two spectra that were downloaded, called 'interval0' and 'late_time', and we saved both of them. The common `saveSpectrum()` function has a `spectra` argument (default: 'all') which determined which are saved. Since our `udg.getSpectra()` calls that common function behind the scenes, we can give it the `spectra` argument if we want, like this:

In [ ]:
udg.getSpectra(
    GRBName="GRB 130925A",
    saveData=True,
    saveImages=True,
    spectra=("late_time",),
    destDir="/tmp/APIDemo_GRB_Spec2",
    extract=True,
    removeTar=True,
    silent=False,
    verbose=True,
)

Don't forget (see the [introduction](#intro)), you can supply `targetID` instead of `GRBName` if you want, and this can be a list/tuple if you want to get multiple objects. If you do this then the `subDirs` argument of `getSpectra()` becomes important: if `True` (the default) then each GRB's data will be saved into a subdirectory which will be the GRB name or targetID (depending on which you called the function with). If it is `False` then the name/targetID will be prepended to the file names.

**Warning** there is one exception to the above: if you set `subDirs=False` and `extract=True` you will get an error. This is because the contents of the tar files are basically the same, so they have to be extracted into separate directories. Also, because of the way X-ray spectra work, various file names are embedded in other files, so we can't really rename them.

Let's do a quick demo of getting multiple spectra:


In [ ]:
udg.getSpectra(
    GRBName=("GRB 130925A", "GRB 071020"),
    saveData=True,
    saveImages=True,
    destDir="/tmp/APIDemo_GRB_Spec3",
    extract=True,
    removeTar=True,
    silent=False,
    verbose=True,
)

If you look through the output above, you will see that I was not lying: the two GRBs' data were saved in their own subdirectories.

<a id='specDict'></a>
### Storing the spectral data in variables

Let's move on now to the `returnData=True` case. As I told you [earlier](#intro) this will return a `dict` containing the data. All spectra returned by anything in the `swifttools.ukssdc` module have a common structure which I call a "spectrum `dict`", and you can [read about this here](https://www.swift.ac.uk/API/ukssdc/structures.md#the-spectrum-dict). A key thing to note about this structure is that, unlike the light curve `dict` it does not give you the actual spectral data in some Python data structure; instead it gives you the results of the automated spectral fits. The rationale here is that X-ray spectral data cannot simply be manipulated numerically, they need handling through tools such as `xspec`, so they don't really make sense as Python variables and are not likely to be useful to you. Spectral fit results, on the other hand, are very likely to be useful to you, and are just numbers.

So, let's go straight to a demo. First, let's get the data for GRB 130925A again, but this time as a variable only.


In [ ]:
specData = udg.getSpectra(GRBName="GRB 130925A", saveData=False, saveImages=False, returnData=True)

Our `specData` variable is now a spectrum `dict`. I'm not going to spend much time unpacking this because it's already [well documented](https://www.swift.ac.uk/API/ukssdc/structures.md#the-spectrum-dict), but let's give you a bit of help. Generally, I imagine that what you are going to want to access are the spectral fit parameters for a specific spectrum, and if you don't fancy ploughing through [the definition of this data structure](https://www.swift.ac.uk/API/ukssdc/structures.md#the-spectrum-dict) then I'll be nice and save you some effort. Let's see what happened for the late-time spectrum fit to PC data. I know that a power-law will have been fitted to it, because that's all that GRBs are fitted with, so I can go straight to the right part of my variable:

In [ ]:
specData["late_time"]["PC"]["PowerLaw"]

And you see that what we had was a `dict` with all the fit parameters. Obviously, I can actually get at specific values as well:

In [ ]:
specData["late_time"]["PC"]["PowerLaw"]["Gamma"]

I am not going to explore `specData` further here, because of the much-mentioned [dedicated documentation](https://www.swift.ac.uk/API/ukssdc/structures.md#the-spectrum-dict), but let's quickly explicitly see what happens if we ask for more than one spectrum:

In [ ]:
specData = udg.getSpectra(
    GRBName=["GRB 060729", "GRB 070616", "GRB 130925A"],
    returnData=True,
    saveData=False,
    saveImages=False,
)
specData.keys()

As I trust you expected (if you read the [introduction](#intro)), we now have an extra layer tagged onto the front of our `dict`, and because I used the `GRBName` argument in my function call, the keys of this are the GRB names. How we go about accessing a specific spectral fit property should be obvious, but in case not:

In [ ]:
specData["GRB 130925A"]["late_time"]["PC"]["PowerLaw"]["Gamma"]

The last thing to make explicit here is the point that the `getSpectra()` function doesn't count how many entries there are in the `GRBName` (or `targetID`) parameter, just whether it is a string/int, or a list/tuple. So if you supply a tuple (or list) with just one entry, you still get this extra layer in the `dict`, albeit only with one entry:

In [ ]:
specData = udg.getSpectra(
    GRBName=("GRB 060729",),
    returnData=True,
    saveData=False,
    saveImages=False,
)
specData.keys()

And if we'd done this with `saveData=True` then the data would have been saved in the "GRB 060729" subdirectory (or had "GRB 060729" prepended to the filename, if we said `subDirs=False`).

### The third way - save from a variable

As with the light curves, there is a third way&dagger; to use the data, you can pull the data into a variable, and then use that to request the files be saved to disk. This option is provided in case you want to filter your set of GRBs before saving (e.g. maybe you wanted to get a dozen spectra, identify those where the intrinsic NH was &lt;10<sup>21</sup> cm<sup>-2</sup>, and then save the spectral files for those).

We do this by calling `getSpectra(returnData=True)` to get the spectral fits, and then we use the `saveSpectra()` function to decide which to save. The arguments to this are essentially the same as when we called `getSpectra(saveData=True)`, indeed the back-end is the  module-level `saveSpectrum()` function, alluded to earlier and [documented here](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#savespectrum). The one, very important, addition is the argument `whichGRBs`. This can either be 'all' (the default) or a list/tuple of which GRBs' spectra to save. The entries in this list/tuple should be valid keys in the `specData` variable we pass to the function. 

This is all a bit abstract, but it will all become clear (I hope) with the following example:

(&dagger; For Cirith Ungol-related humour, you *will* have to read the light curve section.)

In [ ]:
# Get the data for 3 GRBs in to the `specData` variable:
specData = udg.getSpectra(
    GRBName=["GRB 060729", "GRB 070616", "GRB 130925A"],
    returnData=True,
    saveData=False,
    saveImages=False,
)

# In real code there would probably be some stuff here that leads us to deciding
# that we only want the interval0 spectra and only some of the above GRBs, but for this demo
# it's just hard coded.

udg.saveSpectra(
    specData,
    destDir="/tmp/APIDemo_GRBspec3",
    whichGRBs=("GRB 060729", "GRB 130925A"),
    spectra=("interval0",),
    saveImages=True,
    verbose=True,
    clobber=True,
    extract=True,
    removeTar=True,
)

<a id='slice'></a>

## Time-slicing spectra

For GRBs you can request 'time-sliced' spectra, that is, spectra of the GRB created over a specific time interval, or set of intervals. This is essentially the same as [submitting an `XRTProductRequest`](https://www.swift.ac.uk/API/ukssdc/xrt_prods/RequestJob.md) with the GRB details and a set of time slices, except that you have no control over the model to be fitted; this is always the standard GRB model with Galactic absorption, intrinsic absorber (with redshift if available) and a power-law spectrum.

To request time-sliced spectra for a GRB we use the `timesliceSpectrum()` function. This requires the details of the timeslices and the GRB identifier, and then has various other arguments you can add, such as which grades to use and redshift information, if that which the system has for the GRB is not what you want.

The key parameter is `slices`, a `dict` defining the time slices. The keys of this `dict` are the names you want to give to your spectra, the values give the times, and can be in one of two formats:

1. A tuple comprising the time interval(s) and which mode(s) to extract data for.
1. A simple string giving the time interval(s) to extract data over (e.g. `100-400,500-700`).

As, for example:

```
slices = {
    'early': ['100-800', 'WT']
    'mixed': '100-300,500-1000',
}
```

I've deliberately shown both formats above (because you can mix and match). This would request a spectrum called 'early', which covers times 100-800 seconds since T0, and will only use WT-mode data. A second spectrum called 'mixed' will also be created, and that will be made of data collected between 100-300 seconds and 500-1000 seconds after T0.

You may have noticed that the second option, the string, doesn't give you the ability to request a specific mode (in fact, even using the first option the "mode" entry is optional); so what mode is used? The answer to that lies in the `mode` argument to `timesliceSpectrum()`; this can be 'PC', 'WT' or 'both' (default: 'both') and this is used when no mode is specified.

Let's do an actual demo to explore this properly. I think it reads a bit better if I define the `slices` variable outside the function call, so I will.


In [ ]:
slices = {
    "early": ["100-800", "WT"],
    "mixed": "100-300,500-1000",
}

JobID = udg.timesliceSpectrum(targetID="00635887", slices=slices, verbose=True)

As with the light-curve rebinning, this returns the JobID which we need to retain if we are to do anything. One of the things we can do is to cancel the job, which I won't do here (you can if you want, just uncomment the command) but I'll show you how:

In [ ]:
# udg.cancelTimeslice(JobID)

And as for rebinning, this returns a `bool` telling whether it succeeded or not. We can also check the job status, which is a bit more useful:

In [ ]:
udg.checkTimesliceStatus(JobID)

Or just whether it is complete:

In [ ]:
udg.timesliceComplete(JobID)

While the commands above - deliberately - look like those for rebinning light curves, time-slicing spectra takes a bit longer so you may need to go and make a drink, then come back and try the cell above again until it returns `True`.

Now that it is true, we can get at the data, and in this case we can actually use exactly the same function as above - `getSpectra()`, but instead of giving a `GRBName` or `targetID` we give a `JobID`. Beyond that, the function looks and behaves exactly as above - because it is the same function! I will call the function now and both return the data *and* save it to disk:

In [ ]:
specData = udg.getSpectra(
    JobID=JobID,
    returnData=True,
    saveData=True,
    saveImages=True,
    destDir="/tmp/APIDemo_slice_spec",
    extract=True,
    removeTar=True,
    silent=False,
    verbose=True,
)

`specData` is a spectral `dict` as before, but let's have a quick look at our newly-made spectrum:

In [ ]:
specData["early"]["WT"]

As you can see, the data were fitted here with an absorber at redshift 0.593; this is because that redshift has been recorded for the GRB in our (UKSSDC) GRB system. Maybe you think this is wrong, or want to fit without a redshifted absorber, you can do that by supplying the `redshift` parameter to `timesliceSpectrum()`.

By default this is `None` (i.e. the Python entity `None`) which means "Use whatever you have stored in the GRB system". You can supply either a redshift value to use, or the **string** 'NONE' which means "Do not use a redshift". i.e.

In [ ]:
# Uncomment the line you want to try:
# JobID = udg.timesliceSpectrum(targetID='00635887', slices=slices, redshift=2.3)
# JobID = udg.timesliceSpectrum(targetID='00635887', slices=slices, redshift='NONE')


You can then check the status and get the spectrum as above, and you will find that either no redshift was applied to the absorber, or a redshift of 2.3, depending which one you tried.

----


<a id='ban'></a>
## Burst analyser

This API gives access to all the data in the burst analyser, with some enhancements to which I will return in a moment. First, a reminder that this webpage is documenting the API, not the burst analyser, so if you don't understand some of what I'm discussing, I advise you to look at the [burst analyser paper](https://ui.adsabs.harvard.edu/abs/2010A%26A...519A.102E/abstract) and/or [online documentation](https://www.swift.ac.uk/burst_analyser/docs.php). Surprisingly enough, we get at burst analyser data with the function: `getBurstAnalyser()`. The burst analyser is built on top of light curves, and I will in places refer to things in the [light curve section](#light-curves) so it may be advisable to read that before this.

The burst analyser is a rather complex beast, because it has so many different datasets in it. We have 3 instruments, multiple energy bands, unabsorbed and observed fluxes, hardness ratios, inferred photon indices and energy conversion factors... oh yes, and a few different options for the BAT binning and multiple UVOT filters. All in all, it's complicated. On the website this is all managed through dividing the page into sections and giving various controls. For the API, it's handled by defining a data structure, the burst analyser `dict`, that contains everything, allowing you to explore it. This does mean that there are an awful lot of parameters available for us to consider when saving burst analyser data.

We will return to the burst analyser `dict`, and those parameters, in a minute, but first let's discuss the concept of saving data, because for the burst analyser this is a little more complicated than for the above products.

Conceptually, the situation is exactly the same as for light curves and spectra: you can either get the files from the website and save them straight to disk, or you can download them into a `dict` and, if you want to, write files to disk based on that. However, the way the files are organised for the website is optimised for online visualisation rather than access and manipulation, whereas the whole point of the API *is* access and manipulation. As a result, I decided to prioritise usefulness over uniformity, and give `getBurstAnalyser()` *three* options for what it does:

* `returnData` - (default `False`), whether or not the function should return the burst analyser `dict`.
* `saveData` - (default: `True`), whether the data in the burst analyser `dict` to be saved to disk.
* `downloadTar` - (default: `True`), whether the burst analyser tar file should be saved to disk.

Here `saveData` is effectively "the third way" defined for light curves and spectra, but automated: the data are downloaded into a burst analyser `dict` and then saved from that (that `dict` is, however, discarded unless `returnData=True`), but the files saved are, I think, much more helpful than those you would get just by grabbing the files from the website. `downloadTar` does let you pull the files straight from the web, and has accompanying boolean arguments `extract` and `removeTar` which lets you, er, extract the data from the `tar` file and then remove said file. I haven't included an explicit demonstration of `downloadTar=True` here because it should be obvious what I mean, it's easy for you to test, and it creates a *lot* of files; *and* because I personally advocate `saveData=True` instead. Oh, and as with the other products, these three parameters are not mutually exclusive, then can all be `True` (or `False` bur I still can't see why you would do that).

Before we plunge into some demos, though, I should elaborate briefly on the above: what are the 'enhancements' I referred to, and the difference between the files that you get with `downloadTar` and `saveData`? There are two parts to this.

First: most of the light curves in the burst analyser data actually consist of three time series: the flux (i.e. the light curve), the photon index and the energy conversion factor (ECF), and on the website (and in the downloable `tar` file) these are all in separate files, even though they share a common time axis. So if you want to do any manipulation, you have to read multiple files and then join them on said time axis. With `saveData=True`, this is done for you, so for each light curve you get **one** file that has columns for flux, photon index, ECF etc.

Second: The issue of error propagation for the burst analyser is complicated (do see [burst analyser paper](https://ui.adsabs.harvard.edu/abs/2010A%26A...519A.102E/abstract) and/or [online documentation](https://www.swift.ac.uk/burst_analyser/docs.php) if you want details). As the documentation explains, in the light curves online (and in the `tar` file), the errors on the flux values are derived solely from the error on the underlying count-rate light curves, the uncertainty in the spectral shape and hence flux conversion are **not** propagaged into those errors. The reasons for this are subtle (but important, and discussed in the documentation), and of course you can do this propagation yourself if you grab all the files. However, in the API, we do this for you! The downloaded data (which we will explore soon) contain two sets of errors &#8212; with and without propagation &#8212; and you can choose which to save to disk as we shall demonstrate in a second.

Right, that's enough talk, let's get to work.

<a id='ban_dict'></a>
### Getting the burst analyser data into a variable

I'm going to begin the burst analyser tutorial with the `returnData=True` case (unlike for the earlier products) because this introduces the data that we save to disk with `saveData=True`.

As with all previous products, this needs the `GRBName` or `targetID` arguments (see [the introduction](#intro)) which can be single values or lists/tuples, and it returns a `dict`. Unlike the light curve and spectral `dict`s, the burst analyser `dict` only appears for GRBs, and so while it is described in [the data structure documentation](https://www.swift.ac.uk/API/ukssdc/structures.md#the-burst-analyser-dict) it is only touched on lightly and I will give a full demonstration here.

The burst analyser `dict` is not too complicated, and in concept is intentionally reminscent of the way the spectral and light curve `dict`s were built. The burst analyser `dict` has (up to three) layers:

* Instruments
 * [BAT binning & HR Data]
   * Light curves & HR Data
   
For obvious reasons, the middle layer is only present for the BAT data, and the different instruments have slighly different contents as we'll see. 

You can see a detailed schematic in [the data structure documentation](https://www.swift.ac.uk/API/ukssdc/structures.md#the-burst-analyser-dict) but let's instead here explore interatively. First, let's get a single GRB:

In [ ]:
data = udg.getBurstAnalyser(GRBName="GRB 201013A", returnData=True, saveData=False)

Right, now we can explore `data`. The top level of this `dict` is all about the instruments:

In [ ]:
data.keys()

If you've followed any of the other data structures you can probably guess what this means. The 'Instruments' entry is a list, telling us what intstruments' data we have; the other entries are all the `dict`s containing those data, obviously indexed by the instrument, so:

In [ ]:
data["Instruments"]

should not be a surprise.

You will note that, as for the website, the BAT data, and the BAT data without spectral evolution are separate. I spent a while looking into putting them both inside the same entry and then decided it was much more sensible to keep them separate. We'll explore these data, one instrument at a time. The details of this `dict` differ slightly for each instrument so we'll go through them separately:

#### BAT data

As you can see from the description of the overall structure of the burst analyser `dict`, the BAT data should contain a level which divides the data according to how the BAT data were binned:

In [ ]:
list(data["BAT"].keys())

It may not be immediately obvious with all the keys, but again this is the same design as the higher level and in other contexts: we have the key 'Binning', which is a list of the different binning methods available, which themselves exist as keys in this `dict`. There is also an entry `HRData` which we will start off with.

The burst analyser works by taking a hardness ratio time series and using it to infer the spectral properties and ECF at a given time. The hardness ratio is created with a fixed binning (and then we interpolate), which is why `HRData` appears at this level. It is simply a `DataFrame` like a light curve, and contains a lot of columns; the hardness ratio and then the various ECFs (to the different energy bands the burst analyser light curves are in) and the photon index. 

Let's look at it:

In [ ]:
data["BAT"]["HRData"]

There's not much more to say really, so let's turn our attention to the 'Binning' list:

In [ ]:
data["BAT"]["Binning"]

I trust the contents of this aren't a particular shock. One thing you may be wondering about is why, for the SNR (signal-to-noise ratio) binning there are two sets, some ending "\_sinceT0". This is just because you get somewhat different results with the SNR-binning approach if you consider only data taken at t>0 (necessary for plotting a logarithmic time axis) or all data, so these are available separately.

Each of these entries is itself a `dict`, taking us to the next level of the burst analyser `dict`, so let's pick one as an example:

In [ ]:
data["BAT"]["SNR4"].keys()

This is just a light curve `dict` ([documented here](https://www.swift.ac.uk/API/ukssdc/structures.md#the-light-curve-dict)); 'Datasets' will list the light curves present, and the other keys are those light curves.

There are no 'Binning' or 'TimeFormat' keys because for the burst analyser everything is in seconds since T0, and the binning was set by which `dict` we're in. Let's just check the contents:

In [ ]:
data["BAT"]["SNR4"]["Datasets"]

In [ ]:
data["BAT"]["SNR4"]["Density"]

You will note what I have said in the introduction above: the flux, photon index and ECF are all in this one table, and there are two sets of flux errors, with and without the ECF errors propagated. There is also a column, 'BadBin'. Some BAT data on the burst analyser are flagged as 'bad' ('unreliable' may be a better word, with hindsight) - you can read the burst analyser docs to find out why. On the website a checkbox lets you decide whether or not to plot these: in this API, you get all the bins, and the 'BadBin' `bool` column tells you which ones were marked as bad.

You may have noticed that the three different energy bands are in three different light curves. I could have combined these into one curve with lots of columns, but this way seems neater to me. If you really want to combine them yourself then `DataFrame.merge()` is your friend.

Having covered the BAT in detail, we can pass through the other instruments a bit more rapidly.

#### BAT_NoEvolution data

The BAT data without spectral evolution are a bit simpler because, well, they don't account for spectral evolution. So there is no hardness ratio, no photon index and ECF time series. Instead we have a single set of ECFs taken from a spectral fit to the data taken over T90. So, let's have a quick look:

In [ ]:
list(data["BAT_NoEvolution"].keys())

This looks rather like the BAT data, except that there is no 'HRData' entry, and there is an 'ECFs' one. The latter simply gives those ECFs from the T90 spectrum:

In [ ]:
data["BAT_NoEvolution"]["ECFs"]

Beyond this, the BAT_NoEvolution data look just like the BAT data. i.e. if I pick a binning method, say SNR4 (again) and explore it:

In [ ]:
list(data["BAT_NoEvolution"]["SNR4"].keys())

In [ ]:
data["BAT_NoEvolution"]["SNR4"]["Datasets"]

In [ ]:
data["BAT_NoEvolution"]["SNR4"]["Density"]

The only real different here is that this `DataFrame` is much simpler, because we are not accounting for spectral evolution; there is also no propagated ECF error as the ECF comes from the spectrum not the hardness ratio (this doesn't mean it's without error, but the actual burst analyser processing never calculates the ECF error in this case. If I decide it should, then this part of the `dict` will of course be updated, but no existing contents will be changed).

#### XRT data

If you refer [way back up this notebook to the burst analyser `dict` introduction](#ban_dict), you'll remember that the 'binning' layer is only present for BAT, because this is the only instrument for which the burst analyser has multiple binning options. So, when we explore XRT data we should come straight into a light curve `dict`.

In [ ]:
list(data["XRT"].keys())

And we do! Although eagle-eyed readers will realise there is an extra "HRData_PC" entry. This is analogous to the BAT entry, giving the hardness ratio and its conversion to photon index and ECF. The HR data is separated out for the two XRT modes; although this GRB only has PC mode data. For completeness, let's have a quick look at things:

In [ ]:
data["XRT"]["HRData_PC"]

I must point out one little problem here: there should, really, be an 'HRData_incbad' key here, and there isn't, which is why there is only one bin in this hardness ratio. It turns out that this file doesn't exist (the \_incbad hardness ratio exists *and is used to create the burst analyser light curves*, but this nice, combined file of everything isn't saved to disk). I will look into fixing this, and I guess I'll have to remake all the burst analyser data to create all the files(!) which may take a while, but the problem is in the burst analyser, not the API. When I've fixed it, the '\_incbad' entries will appear.

The other things are just the flux light curves, analogous to the BAT one:

In [ ]:
data["XRT"]["Density_PC_incbad"]

#### UVOT data

Lastly, let's check out the UVOT data. This is very simple and like XRT we get straight into a light curve `dict`:

In [ ]:
data["UVOT"].keys()

UVOT data only appear in the 'observed flux in their native bands' plot in the burst analyser, which is why we only have one entry per filter. In some cases there will be upper limits as well as detections, which would give datasets like 'b_UL'.

The UVOT light curves should appear as you expect, but let's look:

In [ ]:
data["UVOT"]["uvm2"]

Yep, nothing surprising there.

#### Other arguments, and a note on memory usage

The above example was a very simple one, in which all of the data for GRB 201013A were retrieved. Sometimes you may not want to get all of the data (why waste time transferring things you don't need?) and indeed, there are cases where you *can't* get all of the data in one go, as it violates the memory resource limits of our web server (try replacing "GRB 201013A" with "GRB 130427A" in the example above and you'll hit this problem). So there are three arguments you can pass to `udg.getBurstAnalyser()` to request only some of the data be retrieved. **I strongly advocate using some of these** if only because I find it hard to believe that you most users will actually want all of the data. These arguments are below. All of them can be the string 'all' (the default if unspecified), or a list/tuple of strings. The contents of these are case insensitive. But what are they?

* `instruments` - which instrument(s) data to get (bat, xrt, uvot).
* `BATBinning` - which methods of BAT data binning to get (snr4, snr5, snr6, snr7, timedel0.004, timedel0.064, timedel1, timedel10)
* `bands` - The energy bands to retrieve (observedflux, density, batband, xrtband)


### Saving the data

Now we've dug through what the returned data look like, we can get to the point of writing them to disk. We can do this in two ways, either by calling `udg.saveBurstAnalyserData(data, **kwargs)`, where the `data` object is the one we created above; or by setting `saveData=True` in the call to `getBurstAnalyserData`. These two things act in exactly the same way behind the scenes, so can be considered identical.

As usual when saving data, you can specify the `destDir`, and the `subDirs` option is `True` by default, behaving exactly as it has done for light curves and spectra. That is (in case you've forgotten while wading through the `dict` above): if you requested a single GRB, it does nothing. If you requested multiple GRBs (or provided a list/tuple with one entry) then it will create a subdirectory per GRB, named either by the GRB name or the targetID, depending on which one you used to select the GRBs. If `subDirs` was `False` and you supplied a list/tuple of GRBs, the GRB name or targetID will be prepended to the file names.

For the burst analyser there are a few extra options you can specify when saving data: these are the same whichever of the two methods above you use, because behind the scenes they both call `udg.saveSingleBurstAn()` and just pass `**kwargs` to it. There are quite a few, and I'll discuss the key ones in a moment, or you can read all about them by executing the following cell.

In [ ]:
help(udg.saveSingleBurstAn)

Things like `asQDP`, `incbad`, and `nosys` you should already be familiar with from the light curves, but there are a few new ones, which are quite important, to draw your attention to.

* **instruments** You can select to only download or save certain instruments. This parameter defaults to 'all', but you can supply a list/tuple of some subset of 'BAT', 'XRT' and 'UVOT' (BAT will get 'BAT_NoEvolution' too). This is the one parameter which behaves slightly differently depending on context. If you call `getBurstAnalyser` with the `instruments` parameter, then this affects which data are *downloaded* (and can make a notable difference to execute time), and thus also added to the dict (if `returnData=True`). Whereas when calling `saveBurstAnalyser` you have already downloaded the data, and you are requesting to save only a subset of what you downloaded. (In case it isn't obvious, if you call `getBurstAnalyser(instruments=('XRT',))` and then try (`saveBurstAnalyser(instruments=('BAT',))` you won't save anything because no BAT data have been downloaded.)
* **usePropagatedErrors** As discussed above, this API calculates the flux errors with the ECF errors propagated. If you just save the burst analyser data to a text file, you get both set of errors; but if you specify `asQDP=True` then only one set of flux errors are written. By default, these are those without the ECF errors - as on the website. This option lets you change that.
* **badBATBins** Whether or not BAT data rows with bad (or unreliable) data should be saved (i.e. rows where 'BadBin' is `True`).

So, just for the sake of completeness, let's give an example. First, let's save the data we've just downloaded (uncomment the `verbose` line if you want to see where the files are saved):

In [ ]:
udg.saveBurstAnalyser(
    data,
    destDir="/tmp/APIDemo_burstAn1",
    # verbose=True,
    badBATBins=True,
)

But of course, we could have done this without first pulling the data into a variable:

In [ ]:
udg.getBurstAnalyser(
    GRBName="GRB 201013A",
    saveData=True,
    returnData=False,
    destDir="/tmp/APIDemo_burstAn2",
    badBATBins=True,
    # verbose=True
)

Exercise for reader: confirm that these two have produced exactly the same files.

#### Saving the data *and* the tar file

I said I wasn't going to demonstrate getting the tar file, but I do one to make one little point. If you run this (and don't feel obliged to):

In [ ]:
udg.getBurstAnalyser(
    GRBName="GRB 201013A",
    saveData=False,
    returnData=False,
    downloadTar=True,
    extract=True,
    removeTar=True,
    destDir="/tmp/APIDemo_burstAn3",
)

And then this:

In [ ]:
udg.getBurstAnalyser(
    GRBName="GRB 201013A",
    saveData=True,  ### This line has changed compared to the last cell
    returnData=False,
    downloadTar=True,
    extract=True,
    removeTar=True,
    destDir="/tmp/APIDemo_burstAn4",
)

Then if you compare the two directories created by the last two cells, you will notice that the second one has a subdirectory `fromTar`, and the tar file has been extracted there. This is just because if you extract the `tar` file in the same place as you save the data directly, it's frankly a mess, so things are kept separate for you.

In all of these examples I've downloaded a single GRB and done so by name, but as with every other product, you can supply a list/tuple of names, or use targetIDs instead.

<a id='positions'></a>
## Positions

Positions are *so* much simpler than everything above! There are very few options to worry about, and a really simple return strucutre. Positions come with no files, just a simple `dict` of positions. Let's take a peek:

In [ ]:
pos = udg.getPositions(GRBName="GRB 080319B")

In [ ]:
pos

Well, that was easy. For GRBs there are a bunch of positions produced, and by default this function will return all of them, with `None` for missing things. The only thing to note is that `Best` doesn't necessarily mean "has the smallest error", rather it in terms of precedence. Enhanced positions are preferred, if there is none then the Standard position is given, failing that the SPER or then Onboard.

The only extra argument this function has (apart from the usual `silent` and `verbose`) is `positions`. This defaults to 'all', but can instead be a list/tuple of which positions you want to get, of the set above.  And of course, we can request multiple GRBs in one go if we want, as with the other products. To show these both in one go:

In [ ]:
pos = udg.getPositions(GRBName=("GRB 080319B", "GRB 101225A"), positions=("Enhanced", "SPER"))
pos

And, of course, as with everything in here, we could have supplied `targetID` instead of `GRBName` if we wanted.

<a id='obs'></a>
## Obs Data

Phew, nearly at the end of this module. There is one last bit of functionality and we are going to deal with this one really, really quickly. 

If you want to download all of the actual obs data for a GRB we can do that using the function `getObsData`. This is basically a wrapper around `downloadObsData()` in the parent module, which was described in the [parent module documentation](../data.ipynb). It takes the `GRBName` or `targetID` parameter, exactly like everything else in this notebook, it takes `verbose` and `silent`, like everything else everywhere, and any other parameters are just `**kwargs` passed to `downloadObsData`.

So, one example, only getting a little data to save time:


In [ ]:
udg.getObsData(
    GRBName="GRB 201013A",
    instruments=[
        "XRT",
    ],
    destDir="/tmp/APIDemo_downloadGRB",
    silent=False,
)

Well done, you got to the end of a long tutorial. I hope you found reading it rather less trying than I found writing it, and more to the point, that it was helpful. Don't forget to remove all the `/tmp/APIDemo*` directories we've created!